# Purpose:
- Assume data is searched and loaded (develop this part later)
    - Use data from "Saffron" (721291)
- Run design matrix, per session
- Test attaching ophys data as well (consider memory limit)

In [1]:
from DesignMatrix import DesignMatrix
from comb.behavior_ophys_dataset import BehaviorOphysDataset, BehaviorMultiplaneOphysDataset
from comb.behavior_session_dataset import BehaviorSessionDataset
import os
import glob
from pathlib import Path
import numpy as np
import pandas as pd
import xarray as xr
import json

# notebook dev
%load_ext autoreload
%autoreload 2
%matplotlib inline
import capsule_utils
import load_data
import design_matrix_tools as dmtools
import kernel_tools as ktools

In [2]:
session_name = 'multiplane-ophys_721291_2024-05-16_08-57-00'

data_dir = Path('/root/capsule/data/')
raw_path = data_dir / session_name
processed_path = list(data_dir.glob(f'{session_name}_processed*'))[0]

opids = []
for plane_folder in processed_path.glob("*"):
    if plane_folder.is_dir() and not plane_folder.stem.startswith("nextflow"):
        opid = plane_folder.stem
        opids.append(opid)

bod_list = []
for opid in opids:
    bod = load_data.load_plane_data(session_name, opid=opid)
    bod = capsule_utils.add_trials_to_bod(bod)
    bod_list.append(bod)
bod = bod_list[0]

File rig.json not found in /root/capsule/data/multiplane-ophys_721291_2024-05-16_08-57-00


/comb/src/comb/processing/stimulus/stimulus_processing.py:802: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  is_change = is_change.fillna(False)


File rig.json not found in /root/capsule/data/multiplane-ophys_721291_2024-05-16_08-57-00


/comb/src/comb/processing/stimulus/stimulus_processing.py:802: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  is_change = is_change.fillna(False)


File rig.json not found in /root/capsule/data/multiplane-ophys_721291_2024-05-16_08-57-00


...

/comb/src/comb/processing/stimulus/stimulus_processing.py:802: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  is_change = is_change.fillna(False)


File rig.json not found in /root/capsule/data/multiplane-ophys_721291_2024-05-16_08-57-00


/comb/src/comb/processing/stimulus/stimulus_processing.py:802: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  is_change = is_change.fillna(False)


File rig.json not found in /root/capsule/data/multiplane-ophys_721291_2024-05-16_08-57-00


/comb/src/comb/processing/stimulus/stimulus_processing.py:802: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  is_change = is_change.fillna(False)


In [3]:
def build_input_kernels():
    kernels = {
        'intercept':    {'feature':'intercept',   'type':'continuous',    'length':0,     'offset':0,     'num_weights':None, 'dropout':True, 'text': 'constant value'},
        'hits':         {'feature':'hit',         'type':'discrete',      'length':1.5,   'offset':0,    'num_weights':None, 'dropout':True, 'text': 'lick to image change'},
        'misses':       {'feature':'miss',        'type':'discrete',      'length':1.5,   'offset':0,    'num_weights':None, 'dropout':True, 'text': 'no lick to image change'},
        'passive_change':   {'feature':'passive_change','type':'discrete','length':1.5,   'offset':0,    'num_weights':None, 'dropout':True, 'text': 'passive session image change'},
        #'hits':         {'feature':'hit',         'type':'discrete',      'length':.75,   'offset':0,    'num_weights':None, 'dropout':True, 'text': 'lick to image change'},
        #'misses':       {'feature':'miss',        'type':'discrete',      'length':.75,   'offset':0,    'num_weights':None, 'dropout':True, 'text': 'no lick to image change'},
        #'passive_change':   {'feature':'passive_change','type':'discrete','length':.75,   'offset':0,    'num_weights':None, 'dropout':True, 'text': 'passive session image change'},
        #'post-hits':    {'feature':'hit',         'type':'discrete',      'length':1.5,   'offset':0.75,    'num_weights':None, 'dropout':True, 'text': 'lick to image change'},
        #'post-misses':  {'feature':'miss',        'type':'discrete',      'length':1.5,   'offset':0.75,    'num_weights':None, 'dropout':True, 'text': 'no lick to image change'},
        #'post-passive_change': {'feature':'passive_change','type':'discrete','length':1.5,   'offset':0.75,    'num_weights':None, 'dropout':True, 'text': 'passive session image change'},
        'omissions':        {'feature':'omissions',   'type':'discrete',  'length':1.5,      'offset':0,     'num_weights':None, 'dropout':True, 'text': 'image was omitted'},
        #'omissions':        {'feature':'omissions',   'type':'discrete',  'length':0.75,      'offset':0,     'num_weights':None, 'dropout':True, 'text': 'image was omitted'},
        #'post-omissions':   {'feature':'omissions',   'type':'discrete',  'length':2.25,   'offset':0.75,  'num_weights':None, 'dropout':True, 'text': 'images after omission'},
        'each-image':   {'feature':'each-image',  'type':'discrete',      'length':0.75,  'offset':0,     'num_weights':None, 'dropout':True, 'text': 'image presentation'},
        'running':      {'feature':'running',     'type':'continuous',    'length':2,     'offset':-1,    'num_weights':None, 'dropout':True, 'text': 'normalized running speed'},
        # 'pupil':        {'feature':'pupil',       'type':'continuous',    'length':2,     'offset':-1,    'num_weights':None, 'dropout':True, 'text': 'Z-scored pupil diameter'},
        'licks':        {'feature':'licks',       'type':'discrete',      'length':2,     'offset':-1,    'num_weights':None, 'dropout':True, 'text': 'mouse lick'},
        #'false_alarms':     {'feature':'false_alarm',   'type':'discrete','length':5.5,   'offset':-1,    'num_weights':None, 'dropout':True, 'text': 'lick on catch trials'},
        #'correct_rejects':  {'feature':'correct_reject','type':'discrete','length':5.5,   'offset':-1,    'num_weights':None, 'dropout':True, 'text': 'no lick on catch trials'},
        #'time':         {'feature':'time',        'type':'continuous',    'length':0,     'offset':0,    'num_weights':None,  'dropout':True, 'text': 'linear ramp from 0 to 1'},
        #'beh_model':    {'feature':'beh_model',   'type':'continuous',    'length':.5,    'offset':-.25, 'num_weights':None,  'dropout':True, 'text': 'behavioral model weights'},
        #'lick_bouts':   {'feature':'lick_bouts',  'type':'discrete',      'length':4,     'offset':-2,   'num_weights':None,  'dropout':True, 'text': 'lick bout'},
        #'lick_model':   {'feature':'lick_model',  'type':'continuous',    'length':2,     'offset':-1,   'num_weights':None,  'dropout':True, 'text': 'lick probability from video'},
        #'groom_model':  {'feature':'groom_model', 'type':'continuous',    'length':2,     'offset':-1,   'num_weights':None,  'dropout':True, 'text': 'groom probability from video'},
    }
    ## add face motion energy PCs
    # for PC in range(5):
    #     kernels['face_motion_PC_{}'.format(PC)] = {'feature':'face_motion_PC_{}'.format(PC), 'type':'continuous', 'length':2, 'offset':-1, 'dropout':True, 'text':'PCA from face motion videos'}
    
    return kernels

In [4]:
run_params = {'data_type': 'filtered_events'}
input_kernel_dict = build_input_kernels()


response_list = []
run_params_list = []
for bod in bod_list:
    run_params = ktools.process_kernels(input_kernel_dict, run_params, bod)
    response, run_params = load_data.extract_and_annotate_ophys_plane(bod, run_params)
    response_list.append(response)
    run_params_list.append(run_params)

Using filtered events traces
Interpolating neural signal onto stimulus aligned timestamps
Using filtered events traces
Interpolating neural signal onto stimulus aligned timestamps
Using filtered events traces
Interpolating neural signal onto stimulus aligned timestamps
Using filtered events traces
Interpolating neural signal onto stimulus aligned timestamps
Using filtered events traces
Interpolating neural signal onto stimulus aligned timestamps
Using filtered events traces
Interpolating neural signal onto stimulus aligned timestamps
Using filtered events traces
Interpolating neural signal onto stimulus aligned timestamps
Using filtered events traces
Interpolating neural signal onto stimulus aligned timestamps


In [13]:
# assert all run_params are the same
assert all([run_params == run_params_list[0] for run_params in run_params_list])
timestamps = [r['timestamps'] for r in response_list]
assert all([(ts == timestamps[0]).all() for ts in timestamps])
timebins = [r['time_bins'] for r in response_list]
assert all([(tb == timebins[0]).all() for tb in timebins])
ophys_frame_rates = [r['ophys_frame_rate'] for r in response_list]
assert ~np.diff(ophys_frame_rates).any()

In [14]:
# run params should be the same across planes
run_params = run_params_list[0]
run_params['input_kernel_dict'] = input_kernel_dict  # add for future use

# response session array from concatenating all planes
response_session_arr = xr.concat([r['response_arr'] for r in response_list], dim='cell_roi_id')

# set up a response dictionary for adding kernels
response = {}
response['response_arr'] = response_session_arr
response['timestamps'] = timestamps[0]
response['time_bins'] = timebins[0]
response['ophys_frame_rate'] = ophys_frame_rates[0]

# response archive for saving
response_archive = [r['stimulus_interpolation'] for r in response_list]

# response matrix for saving
response_matrix = response['response_arr']

# response info dictionary for saving
response_info = {}
response_info['timestamps'] = response['timestamps']
response_info['time_bins'] = response['time_bins']
response_info['ophys_frame_rate'] = response['ophys_frame_rate']

In [15]:
# add kernels to the designMatrix class
design = DesignMatrix(response['timestamps'], response['ophys_frame_rate'])
dmtools.add_kernels(design, run_params, bod, response) # need the trace array for kernels like population mean or PC1. 

    Adding kernel: intercept
    Adding kernel: hits
    Adding kernel: misses
    Adding kernel: omissions
    Adding kernel: running
                 : Mean Centering
                 : Standardized to unit variance
    Adding kernel: licks
    Adding kernel: im000
    Adding kernel: im031
    Adding kernel: im035
    Adding kernel: im045
    Adding kernel: im054
    Adding kernel: im073
    Adding kernel: im075
    Adding kernel: im106


In [16]:
# assertion
X = design.get_X()
assert X.shape[0] == response['response_arr'].shape[0]

In [88]:
# # Trim based on the kernel offsets
# # Do this during fitting, not here, because there will be a lot of response matrices with offset differences.
# offsets = [int(w.split('_')[1]) for w in X.weights.values]
# min_offset = min(min(offsets), 0)
# max_offset = max(max(offsets), 0)
# use_indices = range(max_offset, X.shape[0] + min_offset)  # min_offset is negative or 0



# Save results
- Save them to scratch folder for reuse.
    - Consider dynamic saving (search and save if not searched) with dataAssets later.
    - Try individual dataAsset conversion (manually).
- Design matrix and response matrix should be saved separately.
    - Design matrix results should have info about the final design matrix
    - Versioning with incremental integers for now. 
        - Same version should have the same set of features.
        - Even though initial feature request is the same, depending on the availability the results can vary
        - If the difference is only with kernel length, we can load previous design matrix and change them directly.
        - Think about this later.
    - Design matrix results:
        - run_params
        - design matrix
    - Response matrix results:
        - Have dff, events, filtered_events in the folder name.
        - response matrix
        - response dictionary (combined across planes)
        - response archive list of dictionary (across planes)

[False, False, False, False, True, True, False]

In [26]:
save_path = Path('/root/capsule/scratch/')

session_save_path_desmat = save_path / f'design_matrix_{session_name}_00' # 00 just for the test
session_save_path_desmat.mkdir(parents=True, exist_ok=True)

data_type = run_params['data_type']
session_save_path_resmat = save_path / f'response_matrix_{session_name}_{data_type}'
session_save_path_resmat.mkdir(parents=True, exist_ok=True)

# serialize sets in run_params
run_params_keys = list(run_params.keys())
set_inds = np.where([type(run_params[key])==set for key in run_params_keys])[0]
for i in set_inds:
    key = run_params_keys[i]
    run_params[key] = list(run_params[key])

run_params_fn = session_save_path_desmat / 'run_params.json'
with open(run_params_fn, 'w') as f:
    json.dump(run_params, f, indent=4)

design_matrix_fn = session_save_path_desmat / 'design_matrix.nc'
X.to_netcdf(design_matrix_fn)

response_matrix_fn = session_save_path_resmat / 'response_matrix.nc'
response_matrix.to_netcdf(response_matrix_fn)

response_info_fn = session_save_path_resmat / 'response_info.npz'
np.savez(response_info_fn, response_info)
    
response_archive_fn = session_save_path_resmat / 'response_archive.npz'
np.savez(response_archive_fn, response_archive)